In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from jpasLAEs.utils import flux_to_mag

from paus_utils import w_central

In [ ]:
# Load the photometry catalog
from load_paus_cat import load_paus_cat

field_name = 'W3'
path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}.csv']
cat = load_paus_cat(path_to_cat)

mask_NB_number = (cat['NB_number'] > 39)
cat['flx'] = cat['flx'][:, mask_NB_number]
cat['err'] = cat['err'][:, mask_NB_number]
cat['NB_mask'] = cat['NB_mask'][:, mask_NB_number]
for key in cat.keys():
    if key in ['flx', 'err', 'NB_mask', 'area']:
        continue
    cat[key] = cat[key][mask_NB_number]

stack_nb_ids = np.arange(12, 26 + 1)
synth_BB_flx = np.average(cat['flx'][stack_nb_ids],
                          weights=cat['err'][stack_nb_ids] ** -2,
                          axis=0)

cat['synth_r_mag'] = flux_to_mag(synth_BB_flx, w_central[-4])
cat['r_mag'] = flux_to_mag(cat['flx'][-4], w_central[-4])

N_sources = len(cat['ref_id'])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

mask = cat['sg_flag'] == 1
ax.scatter(cat['r_mag'][mask],
           (cat['synth_r_mag'] - cat['r_mag'])[mask],
           alpha=0.1, s=1, c='C0',
           rasterized=True)
ax.scatter([], [], label='star_flag=1', c='C0')
mask = cat['sg_flag'] == 0
ax.scatter(cat['r_mag'][mask],
           (cat['synth_r_mag'] - cat['r_mag'])[mask],
           alpha=0.1, s=1, c='C1',
           rasterized=True)
ax.scatter([], [], label='star_flag=0', c='C1')

ax.set_xlim(15, 26)
ax.set_ylim(-1, 2)

ax.legend(fontsize=12)
ax.set_title(field_name)

ax.axhline(0, ls='--', c='k')

ax.set_ylabel(r'$r_\mathrm{synth.}-r$', fontsize=18)
ax.set_xlabel(r'$r$', fontsize=18)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

delta_r_bins = np.linspace(-1.5, 2.5, 50)

r_mask = cat['r_mag'] < 24

mask = (cat['sg_flag'] == 1) & r_mask
to_hist = (cat['synth_r_mag'] - cat['r_mag'])[mask]
ax.hist(to_hist, delta_r_bins,
        color='C0', histtype='step', label='star_flag=1')
print(f'star_flag=1 median: {np.nanmedian(to_hist):0.3f}')

mask = (cat['sg_flag'] == 0) & r_mask
to_hist = (cat['synth_r_mag'] - cat['r_mag'])[mask]
ax.hist((cat['synth_r_mag'] - cat['r_mag'])[mask], delta_r_bins,
        color='C1', histtype='step', label='star_flag=0')
print(f'star_flag=0 median: {np.nanmedian(to_hist):0.3f}')


ax.legend(fontsize=12)
ax.set_title(f'PAUS {field_name}')

ax.set_xlabel(r'$r_\mathrm{synth.}-r_\mathrm{CFHT}$', fontsize=18)
ax.set_ylabel('N sources', fontsize=18)

plt.show()

In [ ]:
from jpasLAEs.load_jpas_catalogs import load_minijpas_jnep
from jpasLAEs.utils import w_central as w_central_jpas

# Load miniJPAS
mj_cat_dir = '/home/alberto/cosmos/LAEs/csv'
mj_cat = load_minijpas_jnep(mj_cat_dir)

stack_nb_ids = np.arange(20, 31 + 1)
synth_BB_flx = np.average(mj_cat['pm_flx'][stack_nb_ids],
                          weights=mj_cat['pm_err'][stack_nb_ids] ** -2,
                          axis=0)

mj_cat['synth_r_mag'] = flux_to_mag(synth_BB_flx, w_central_jpas[-2])
mj_cat['r_mag'] = flux_to_mag(mj_cat['pm_flx'][-2], w_central_jpas[-2])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

delta_r_bins = np.linspace(-1.5, 2.5, 50)

r_mask = mj_cat['r_mag'] < 22

mask = (mj_cat['starprob'] > 0.5) & r_mask
to_hist = (mj_cat['synth_r_mag'] - mj_cat['r_mag'])[mask]
ax.hist(to_hist, delta_r_bins,
        color='C0', histtype='step', label='star_flag=1')
ax.axvline(np.nanmedian(to_hist), ls='--', c='k')
print(f'star_flag=1 median: {np.nanmedian(to_hist):0.3f}')

mask = (mj_cat['starprob'] <= 0.5) & r_mask
to_hist = (mj_cat['synth_r_mag'] - mj_cat['r_mag'])[mask]
ax.hist((mj_cat['synth_r_mag'] - mj_cat['r_mag'])[mask], delta_r_bins,
        color='C1', histtype='step', label='star_flag=0')
ax.axvline(np.nanmedian(to_hist), ls='--', c='k')
print(f'star_flag=0 median: {np.nanmedian(to_hist):0.3f}')


ax.legend(fontsize=12)

ax.set_xlabel(r'$r_\mathrm{synth.}-r_\mathrm{J-PAS}$', fontsize=18)
ax.set_title('miniJPAS')

plt.show()